In [ ]:
from pointcept.engines.defaults import  default_setup
from pointcept.engines.defaults import (
    default_argument_parser,
    default_config_parser,
)
from pointcept.engines.train import TRAINERS
import sculpting
import torch
import numpy as np

/opt/conda/lib/python3.11/site-packages/spconv/pytorch/functional.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  _TORCH_CUSTOM_FWD = amp.custom_fwd(cast_inputs=torch.float16)
/opt/conda/lib/python3.11/site-packages/spconv/pytorch/functional.py:96: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @_TORCH_CUSTOM_BWD
/opt/conda/lib/python3.11/site-packages/spconv/pytorch/functional.py:162: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @_TORCH_CUSTOM_BWD
/opt/conda/lib/python3.11/site-packages/spconv/pytorch/functional.py:242: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @_TORCH_CUSTOM_BWD
/opt/conda/lib/python3.

In [2]:
args = default_argument_parser().parse_args(
    "--config-file /workspaces/Pointcept/configs/scannet/pipeline-trimming-spunet.py --num-gpus 0 --options num_workers=0 num_worker_per_gpu=0 batch_size=1".split()
)
cfg = default_config_parser(args.config_file, args.options)

In [3]:
cfg = default_setup(cfg)
#cfg.num_worker_per_gpu=0
trainer = TRAINERS.build(dict(type=cfg.train.type, cfg=cfg))

[2025-06-19 18:22:22,106 INFO train.py line 136 37994] => Loading config ...
[2025-06-19 18:22:22,107 INFO train.py line 138 37994] Save path: exp/default
[2025-06-19 18:22:22,183 INFO train.py line 139 37994] Config:
weight = None
resume = False
evaluate = True
test_only = False
enable_wandb = False
seed = 21978417
save_path = 'exp/default'
num_worker = 4
batch_size = 1
batch_size_val = None
batch_size_test = None
epoch = 800
eval_epoch = 100
clip_grad = None
sync_bn = False
enable_amp = True
amp_dtype = 'float16'
empty_cache = False
empty_cache_per_epoch = False
find_unused_parameters = False
mix_prob = 0.8
param_dicts = None
hooks = [
    dict(type='CheckpointLoaderAllowMismatch'),
    dict(type='IterationTimer', warmup_iter=2),
    dict(type='InformationWriter'),
    dict(type='SemSegEvaluator'),
    dict(type='CheckpointSaverWandb', save_freq=None)
]
train = dict(type='DefaultTrainer')
test = dict(type='SemSegPredictor', verbose=True)
sculpting_transform = dict(
    type='Trimming

In [4]:
train_loader = trainer.train_loader
i, b = next(enumerate(train_loader))

b

{'coord': tensor([[-0.8293, -0.3002,  0.6190],
         [-0.8285, -0.3073,  0.6231],
         [-0.8345, -0.2824,  0.6094],
         ...,
         [-1.0619,  2.1359,  1.2117],
         [-0.3857,  2.1526,  0.2629],
         [-0.6529,  2.1178, -0.0609]]),
 'grid_coord': tensor([[223,  48,  46],
         [223,  49,  38],
         [223,  49,  37],
         ...,
         [ 32, 178,  39],
         [ 32, 178,  38],
         [ 32, 178,  56]]),
 'segment': tensor([1, 1, 1,  ..., 0, 0, 1]),
 'offset': tensor([120000]),
 'feat': tensor([[0.5857, 0.6227, 0.6940],
         [0.6399, 0.6579, 0.6800],
         [0.6449, 0.6447, 0.7163],
         ...,
         [0.3621, 0.2703, 0.3612],
         [0.3854, 0.3110, 0.1087],
         [0.1937, 0.0849, 0.0356]])}

In [ ]:
import plotly.express as px

coord=b['coord']
px.scatter_3d(x=coord[:,0],y=coord[:,1],z=coord[:,2], color=b['segment'], b['segment'],r=b[:,0],g=b[:,1],b=[:,2])

In [6]:
arrs= [ b["coord"], b["segment"], b['feat']]

In [7]:
[v.shape for v in arrs]

[torch.Size([120000, 3]), torch.Size([120000]), torch.Size([120000, 3])]

In [ ]:
import pandas as pd

c = (1+b['feat'])/2
pd.DataFrame(dict(
    x=b["coord"][:,0],
    y=b["coord"][:,1],
    z=b["coord"][:,2],
    red=c[:,0]*255,
    green=c[:,1]*255,
    blue=c[:,2]*255,
    label=b['segment']
)).to_csv("test.csv")